
## Viasegura
***

### Descripción
Viasegura es una libreria para etiquetar algunos de los atributos de seguridad vial de la metodologia iRAP. Los atributos especificos que pueden ser etiquetados los encontramos en la libreria en la url:

https://github.com/EL-BID/VIAsegura

Hasta el momento hay 15 modelos desarrollados para etiquetar en total 17 atributos (dado que 2 modelos funcionan para dos atributos distintos.

### Alcance

Para ejecutar el sistema y entender el alcance de aplicacion de estos modelos por favor referirse al manual ubicado en la url:

https://github.com/EL-BID/VIAsegura/tree/main/viasegura/manuals

En esa seccion encontraran el contexto general y particular bajo el que se puede utilizar el software
***
***

### Descrição
A Viasegura é uma biblioteca para etiquetar alguns dos atributos de segurança rodoviária da metodologia iRAP. Os atributos específicos que podem ser marcados são encontrados na biblioteca na url:

https://github.com/EL-BID/VIAsegura

Até agora existem 15 modelos desenvolvidos para rotular um total de 17 atributos (já que 2 modelos funcionam para dois atributos diferentes.

### Alcance

Para executar o sistema e entender o escopo de aplicação desses modelos, consulte o manual localizado na url:

https://github.com/EL-BID/VIAsegura/tree/main/viasegura/manuals

Nesta seção você encontrará o contexto geral e particular em que o software pode ser usado.

***
***
### Description
Viasegura is a library to tag some of the road safety attributes of the iRAP methodology. The specific attributes that can be tagged are found in the library at the url:

https://github.com/EL-BID/VIAsegura

So far there are 15 models developed to label a total of 17 attributes (since 2 models work for two different attributes.

### Scope

To run the system and understand the scope of application of these models, please refer to the manual located at the url:

https://github.com/EL-BID/VIAsegura/tree/main/viasegura/manuals

In this section you will find the general and particular context under which the software can be used.

# Setup
***
### Ajuste del Entorno librerias y funciones previas a la ejecucion de los modelos

*Ajuste das bibliotecas e funções do ambiente antes da execução dos modelos.*

*Adjustment of the environment libraries and functions prior to the execution of the models*


### Instalación de librerias

Para esto utilizamos la ultima version disponible en pypi a traves del comando de pip ejecutado directamente en el servidor.

*Instalação de bibliotecas*

*Para isso utilizamos a última versão disponível em pypi através do comando pip executado diretamente no servidor.*

*Installation of libraries*

*For this we use the latest version available in pypi through the pip command executed directly on the server.*

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install viasegura -q
!pip install geopandas matplotlib plotly ipywidgets -q

### Imports

A continuación, tendremos todas los paquetes que seran necesarios para la ejecucion

*A seguir, teremos todos os pacotes que serão necessários para a execução.*

*Next, we will have all the packages that will be necessary for the execution*

In [ ]:
### General
import os
import json
import tempfile
from tqdm.notebook import tqdm
from pathlib import Path

### Viasegura
from viasegura import ModelLabeler, LanesLabeler

### Analyze
import pandas as pd
import numpy as np
import tensorflow as tf

### graphics
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px
from shapely.geometry import LineString, MultiLineString
from shapely.ops import linemerge

### Functions
Definimos algunas funciones genéricas que serviran de ayuda a lo largo de la sesión.

*Definimos algumas funções genéricas que ajudarão ao longo da sessão.*

*We define some generic functions that will help throughout the session*

In [ ]:
### Load images into memory
def load_image(routes):
    imgs = np.array([tf.image.decode_image(tf.io.read_file(str(route))).numpy() for route in tqdm(routes)])
    return imgs


### Layer map
def create_layer(color, gdf_work):
    with tempfile.NamedTemporaryFile(suffix='.json') as tmp_file:
        gpd.GeoDataFrame(gdf_work.loc[gdf_work.color == color].reset_index(drop=True)[['geometry', 'id']]).to_file(
            tmp_file.name, driver="GeoJSON")
        with open(tmp_file.name) as geofile:
            j_file = json.load(geofile)
            i = 0
            for feature in j_file["features"]:
                feature['id'] = str(i).zfill(5)
                i += 1
        layer = {
            'source': {
                'type': "FeatureCollection",
                'features': j_file['features'],
                'name': color,
            },
            'type': 'line',
            'below': 'traces',
            "color": color
        }
        return layer


### Joining multiple linestring
def multi(x):
    lista = list(x)
    return MultiLineString(lista)


### Map creation
def get_graph(data, varia):
    colors_dict = {
        'Present': '#22a800',
        'Adequate': '#22a800',
        'Poor': '#ee0b00',
        'Not present': '#ee0b00',
        'Carriageway of a divided road': '#1ac7b0',
        'Undivided road': '#6817d1',
        'Urban': '#1ac7b0',
        'Rural': '#6817d1',
        'Underdeveloped areas': '#1ac7b0',
        'Unknown': '#6817d1',
        'One': '#1ac7b0',
        'Two': '#6817d1',
        'Three': '#cc8e12'
    }
    variable = varia
    data['color'] = list(map(lambda x: colors_dict.get(x, '#000000'), data[variable].values))
    data['geometry'] = list(
        map(lambda LS, LaS, LE, LaE: LineString([[LS, LaS], [LE, LaE]]), data.longitud_first, data.latitud_first,
            data.longitud_last, data.latitud_last))
    data = gpd.GeoDataFrame(data)
    data['centroid'] = list(map(lambda x: x.centroid, data.geometry.values))
    data['coords'] = list(map(lambda x: x.coords, data.centroid.values))
    data['latitud'] = list(map(lambda x: x[0][1], data.coords.values))
    data['longitud'] = list(map(lambda x: x[0][0], data.coords.values))
    center = {'lat': data.centroid.y.mean(), 'lon': data.centroid.x.mean()}
    data_g2 = gpd.GeoDataFrame(data.groupby(['color']).aggregate({'geometry': lambda x: multi(x)}).reset_index())
    data_g2['geometry'] = list(map(lambda x: linemerge(x), data_g2.geometry.values))
    data_g2['id'] = [str(i).zfill(5) for i in range(len(data_g2))]
    layers = []
    list(tqdm(map(lambda color: layers.append(create_layer(color, data_g2)), list(data_g2.color.unique()))))
    fig_map = px.scatter_mapbox(data,
                                lat='latitud',
                                lon='longitud',
                                # color=variable,
                                hover_data=[variable],
                                opacity=0
                                )
    fig_map.update_layout(
        showlegend=False,
        autosize=True,
        mapbox={
            'style': "carto-positron",
            'center': center,
            'zoom': 12,
            'layers': layers,
        },
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
    )
    return fig_map

# Ejemplo
***

A continuación veremos el primer ejemplo de ejecución

*A seguir veremos o primeiro exemplo de execução*

*Next we will see the first example of execution*

### Inputs

In [ ]:
work_path = Path('./examples')
master_input_frontal = work_path / 'images' / 'frontal'
master_input_lateral = work_path / 'images' / 'lateral'
gps_data_route = work_path / 'gps_info' / 'example.csv'
BATCH_SIZE = 2

### Carga de imagenes
***



In [ ]:
archives_frontal = sorted(os.listdir(master_input_frontal))
archives_frontal_paths = [master_input_frontal/item for item in archives_frontal]
frontal_images = load_image(archives_frontal_paths)

In [ ]:
g = 0
for i in range(5):
    plt.imshow(frontal_images[(g*5)+i])
    plt.show()

In [ ]:
archives_lateral = sorted(os.listdir(master_input_lateral))
archives_lateral_paths = [master_input_lateral/item for item in archives_lateral]
lateral_images = load_image(archives_lateral_paths)

In [ ]:
g = 0
for i in range(5):
    plt.imshow(lateral_images[(g*5)+i])
    plt.show()

In [ ]:
number_groups = frontal_images.shape[0]//5 + (1 if (frontal_images.shape[0]%5)>0 else 0)
print(f'El numero de grupos es de {number_groups}')

## Ejecución del modelo
***

Execução do modelo

Model execution

### Instancia de objetos
A continuacion vamos a instanciar los objetos que permitiran hacer la prueba, estos son los labelers

*Instância do objeto*

*A seguir vamos instanciar os objetos que nos permitirão fazer o teste, estes são os rotuladores.*

*Object instance*

*Next we are going to instantiate the objects that will allow us to do the test, these are the labelers*


- frontal_filters = [delineation, street, carriageway, service_road, road_condition, skid_resistance, upgrade_cost, speed_management, bicycle_facility, quality_of_curve,
vehicle_parking,
property_access_points]
- lateral_filters = [area_type, land_use]
- LanesLabeler = [number_of_lanes]

In [ ]:
# Choose your device here
# device='/device:GPU:0'
device='/device:CPU:0'

# Models loading
system_path='./'
frontallabeler = ModelLabeler(system_path=system_path, model_type = 'frontal', model_filter = ['delineation','street_lighting','carriageway'], device=device)
laterallabeler = ModelLabeler(system_path=system_path, model_type = 'lateral', device=device)
lanes_labeler = LanesLabeler(system_path=system_path, models_device=device)

### Ejecutando el Modelo
A continuación, ejecutamos los tres grupos de modelos utilizando el objeto particular para cada uno

*Em seguida, executamos os três grupos de modelos usando o objeto específico para cada um*

*A continuação, executamos os três grupos de modelos usando o objeto particular para cada um*

In [ ]:
### Frontal Labeler Execution
frontal_results = frontallabeler.get_labels(frontal_images, batch_size = BATCH_SIZE)

In [ ]:
### Lanes Labeler Execution
lanes_results = lanes_labeler.get_labels(frontal_images, batch_size = BATCH_SIZE)

In [ ]:
### Lateral Labeler Execution
lateral_results = laterallabeler.get_labels(lateral_images, batch_size = BATCH_SIZE)

In [ ]:
frontal_results.keys()

In [ ]:
laterallabeler.classes

In [ ]:
frontal_results['classification']

## Resultados
***
Primero agrupamos los resultados obtenidos

*Primero agrupamos os resultados obtidos*

*First we group the results obtained*

In [ ]:
results_df = pd.concat([pd.DataFrame(result['classification']) for result in [frontal_results, lateral_results, lanes_results]], axis=1)
results_df

### Recoleccion de datos GPS
Levantamos la data gps del csv de entrada (pudiera estar en otros formatos, el codigo para levantar esa data depende del formato)

*Levantamos os dados gps do csv de entrada (podemos estar em outros formatos, o código para levantar esa data depende do formato)*

*We get the gps date from the input csv (it could be in other formats, the code to get this date depends on the format)*

In [ ]:
gps_data = pd.read_csv(gps_data_route, sep = ';', decimal = ',')
gps_data['image_number'] = list(map(lambda x: int(x.split('.')[0].split('_')[0]), gps_data.img_cen.values))
gps_data['group'] = list(map(lambda x:(x-1)//5, gps_data['image_number']))
gps_data = gps_data.groupby(['group']).aggregate({'latitud':['first','last'], 'longitud':['first','last']}).reset_index()
gps_data.columns = [col[0] if col[1]=='' else f'{col[0]}_{col[1]}' for col in gps_data.columns]
gps_data



Agrupamos la data gps con los datos de resultados con que tendremos un DataFrame con las ubicaciones (inicio y fin) y los diferentes resultados de ejecutar el modelo

*Agrupamos os dados gps com os dados de resultados que tendem a um DataFrame com as publicações (inicial e final) e os diferentes resultados de execução do modelo*

*We group the gps data with the results data with which we will have a DataFrame with the locations (start and end) and the different results of executing the model*

In [ ]:
results_df = pd.concat([gps_data, results_df ], axis=1)
results_df.to_csv('outputs/results_example.csv', sep='|', decimal='.')

In [ ]:
results_df

### Visualizando los resultados en un mapa
***
*Visualizando os resultados em um mapa*

*Viewing the results on a map*

In [ ]:
map_variable = get_graph(results_df, 'street_lighting')## delineation,	carriageway,	street_lighting,	area_type,	land_use,	number_of_lanes
map_variable